In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Let's Start

### Import the Libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from pylab import rcParams

%matplotlib inline

In [ ]:
df = pd.read_csv("/kaggle/input/szeged-weather/weatherHistory.csv")
df.head()

In [ ]:
df.columns

In [ ]:
df.dtypes

### Handelling The Date Column

In [ ]:
df["Formatted Date"] = pd.to_datetime(df["Formatted Date"], errors='coerce', utc=True)
df.head()

In [ ]:
df.dtypes

In [ ]:
df['Month'] = df['Formatted Date'].dt.month
df['Year'] = df['Formatted Date'].dt.year
df['Day'] = df['Formatted Date'].dt.day
df['Hour'] = df['Formatted Date'].dt.hour
df.head()

In [ ]:
df.describe().T

In [ ]:
df.drop(["Formatted Date", 'Loud Cover', 'Daily Summary'], axis =1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
numerical = df.select_dtypes(include=["float64", "int64"]).keys()
categorical = df.select_dtypes(include=["object"]).keys()
numerical, categorical

### Handeling Categorical Variables

In [ ]:
df_cat = df[categorical]
df_cat.head()

In [ ]:
df_cat.info()

In [ ]:
df_cat["Summary"].value_counts().values

In [ ]:
def unique_vals(df, cols):
    rcParams['figure.figsize'] = 15, 16
    fig,ax = plt.subplots(len(cols))
    for i,col in enumerate(cols):
        sns.countplot(x=df[col],ax=ax[i])
        ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=30, ha='right')

In [ ]:
unique_vals(df_cat, ["Summary", "Precip Type"])

#### Summary Column

In [ ]:
df["Summary"].describe(), df["Summary"].value_counts()

In [ ]:
(df['Summary'].value_counts()/len(df))*100

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df['Summary'] = le.fit_transform(df['Summary'])

In [ ]:
df.head() 

In [ ]:
df.isnull().sum()

In [ ]:
df['Precip Type'].value_counts()

In [ ]:
df = pd.get_dummies(df,columns=['Precip Type'], prefix='Type_')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

### Handeling the outliers

In [ ]:
df_num = df[numerical]
df_num.head()

In [ ]:
df_num.columns

In [ ]:
def outlier_visualizer(df,cols):
    fig,ax = plt.subplots(len(cols),2)
    